# Setup Environment

In [ ]:
!cd /opt && wget -O conda.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh && bash conda.sh -b -f -u && echo PATH="/root/miniconda3/bin:$PATH" > /root/.bashrc

In [ ]:
!source /root/.bashrc && conda init bash

In [ ]:
!source /root/.bashrc && conda update --all

In [ ]:
!source /root/.bashrc && conda create --name tf python=3.9 -y || conda activate tf && conda install -c conda-forge cudatoolkit=11.2 cudnn=8.4.1.50 -y && export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/ && mkdir -p $CONDA_PREFIX/etc/conda/activate.d || echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/' > $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh

In [ ]:
!source /root/.bashrc && conda activate tf && sudo apt update && sudo apt upgrade -y && sudo apt install protobuf-compiler python3-dev python3-pip python-is-python3 -y
!source /root/.bashrc && conda activate tf && pip install --upgrade tensorflow
!source /root/.bashrc && conda activate tf && pip install --upgrade numpy

In [ ]:
!source /root/.bashrc && conda activate tf && sudo apt install zlib1g && pip install gdown && cd /opt && gdown https://drive.google.com/u/0/uc?id=1gYZCQCzfnNYAzlgEHvsb9LGHN_QF2Q1Y && sudo dpkg -i cudnn-local-repo-ubuntu1804-8.5.0.96_1.0-1_amd64.deb && sudo cp /var/cudnn-local-repo-*/cudnn-local-*-keyring.gpg /usr/share/keyrings/ && sudo apt update

In [ ]:
!source /root/.bashrc && conda activate tf && pip install gdown && cd /opt && gdown https://drive.google.com/u/0/uc?id=15HZAdhMGHeofGsJFrguiHjNBYI46iyZb && os="ubuntu1804" && tag="cuda11.6-trt8.4.3.1-ga-20220813" && sudo dpkg -i nv-tensorrt-repo-${os}-${tag}_1-1_amd64.deb && sudo apt-key add /var/nv-tensorrt-repo-${os}-${tag}/*.pub && sudo apt update && sudo apt install tensorrt -y && python -m pip install numpy && gdown https://drive.google.com/u/0/uc?id=12HIi2Ny3voCxCvq9PiMjQ5EEk-zp3hYD && tar -xf TensorRT-8.4.3.1.Linux.x86_64-gnu.cuda-11.6.cudnn8.4.tar.gz && cd TensorRT-8.4.3.1/python && pip install tensorrt-8.4.3.1-cp39-none-linux_x86_64.whl && python -m pip install protobuf && sudo apt install uff-converter-tf -y &&  sudo apt install tensorrt-dev=8.4.3.1-1+cuda11.6 -y && sudo apt-mark hold tensorrt-dev

In [ ]:
import os

if not os.path.isdir("Tensorflow/models"):
    !source /root/.bashrc && conda activate tf && cd Tensorflow && git clone https://github.com/tensorflow/models.git
    
!source /root/.bashrc && conda activate tf && cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

In [ ]:
WORKSPACE_PATH = "Tensorflow/workspace"
SCRIPTS_PATH = "Tensorflow/scripts"
APIMODEL_PATH = "Tensorflow/models"
ANNOTATION_PATH = f"{WORKSPACE_PATH}/annotations"
IMAGE_PATH = f"{WORKSPACE_PATH}/images"
MODEL_PATH = f"{WORKSPACE_PATH}/models"
PRETRAINED_MODEL_PATH = f"{WORKSPACE_PATH}/pre-trained-models"
CUSTOM_MODEL_NAME = "model"
CONFIG_PATH = f"{MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config"
CHECKPOINT_PATH = f"{MODEL_PATH}/{CUSTOM_MODEL_NAME}/checkpoint"


In [ ]:
!source /root/.bashrc && conda activate tf && mkdir {PRETRAINED_MODEL_PATH}/{CUSTOM_MODEL_NAME} || wget -O {PRETRAINED_MODEL_PATH}/{CUSTOM_MODEL_NAME}.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz && tar -xf {PRETRAINED_MODEL_PATH}/{CUSTOM_MODEL_NAME}.tar.gz -C {PRETRAINED_MODEL_PATH}/{CUSTOM_MODEL_NAME} --strip-components 1 && mkdir {MODEL_PATH}/{CUSTOM_MODEL_NAME}


Copy the appropriate configuration file from ``Tensorflow/models/research/object_detection/configs/tf2/`` to ``Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/`` as ``pipeline.config``. It is important to copy the configuration file to the correct directory, otherwise the training will fail.

# 1. Create Label Map and TF Records

In [ ]:
labels = [{"name": "hoop", "id": 1}]

with open(f"{ANNOTATION_PATH}/label_map.pbtxt", "w") as f:
    for label in labels:
        f.write("item { \n")
        f.write(f"\tname:'{label['name']}'\n")
        f.write(f"\tid:{label['id']}\n")
        f.write("}\n")


In [ ]:
!source /root/.bashrc && conda activate tf && python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!source /root/.bashrc && conda activate tf && python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

# 2. Import Required Libraries

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format


# 3. Update the Configuration File

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)


In [ ]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 16
pipeline_config.train_config.fine_tune_checkpoint = (
    f"{PRETRAINED_MODEL_PATH}/{CUSTOM_MODEL_NAME}/checkpoint/ckpt-0"
)
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = f"{ANNOTATION_PATH}/label_map.pbtxt"
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [
    f"{ANNOTATION_PATH}/train.record"
]
pipeline_config.eval_input_reader[
    0
].label_map_path = f"{ANNOTATION_PATH}/label_map.pbtxt"
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    f"{ANNOTATION_PATH}/test.record"
]


In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:
    f.write(config_text)


# 4. Train the Model

In [ ]:
print(
    f"source /root/.bashrc && conda activate tf && python {APIMODEL_PATH}/research/object_detection/model_main_tf2.py --model_dir={MODEL_PATH}/{CUSTOM_MODEL_NAME} --pipeline_config_path={MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config --num_train_steps=30000"
)
